In [1]:
import numpy as np
import math
from itertools import combinations
import copy
np.set_printoptions(suppress=True, linewidth=np.nan)


### Adyacent Graph without cycles


In [2]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [3]:
def minimum_ratio_test_multiple(col: np.ndarray, b: np.ndarray) -> int:
    map_test: dict[float, list[int]] = {}
    min_key = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            test = round(b[k] / col[k], 6)
            if test in map_test:
                map_test[test].append(k)
            else:
                map_test[test] = [k]
            min_key = min(min_key, test)
    return map_test[min_key] if min_key != np.Inf else None


In [4]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    tableau[row] = tableau[row] / tableau[row, col]
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]


In [5]:
def correct_all_tableau(tableau: np.ndarray, basic_var: list[int]):
    # correct basic variable of all column with value distict of zero
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau = pivoting(tableau, row, col)
    print("All tableau corrected")
    return tableau


In [6]:
def basic2vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1 
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    return x[:n]


In [7]:
def real_index(basic_var: list[int]):
    return list(map(lambda x: x + 1, basic_var))


In [8]:
def non_cycle_adjacent_graph(tableau: np.ndarray, basic_var: list[int], row: int = None, col: int = None) -> None:
    vertex = basic2vertex(tableau, basic_var)    
    if (tableau[0, :-1] > 0).any():
        # recursive simplex for each column candidate
        for col in np.where(tableau[0, :-1] > 0)[0]:
            # minimum ratio test
            multi_row = minimum_ratio_test_multiple(col=tableau[1:, col], b=tableau[1:, -1])
            if multi_row != None:
                for row in multi_row:
                    # temp array
                    temp_tableau = np.copy(tableau)
                    temp_basic_var = copy.copy(basic_var)
                    # pivoting
                    pivoting(temp_tableau, row + 1, col)
                    # swap row with col
                    temp_basic_var[row] = col
                    # recursive proccess
                    print(f"vertex = {vertex}, x_B = {real_index(basic_var)}\n{tableau}")
                    non_cycle_adjacent_graph(temp_tableau, temp_basic_var, row, col)
            else:
                print("The model is unbounded")
    else:
        print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}")
        print("end proccess")
        

#### Dovetail Model

In [9]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
non_cycle_adjacent_graph(tableau, basic_var, [])

vertex = [0. 0.], x_B = [3, 4, 5, 6]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
vertex = [6. 0.], x_B = [3, 1, 5, 6]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
vertex = [4.5 4.5]	x_B = [2, 1, 5, 6]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]
end proccess
vertex = [0. 0.], x_B 

### Práctica Dirigida 4

In [10]:
A = [[-2, 6, 1, 0, 0],
     [2, 4., 0, 1, 0],
     [1, 0., 0, 0, 1]]

b = [0, 10, 4.]

c = [-1, 1, 0, 0, 0.]

# un vertice suficiente para visitarlos a todos
basic_var_list = [[0, 3, 2]] # respetar el orden en la tabla

for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     tableau = correct_all_tableau(tableau, basic_var)
     non_cycle_adjacent_graph(tableau, basic_var, [])

------- New simplex proccess -------


TypeError: 'NoneType' object is not subscriptable

### CM4E1 Curso: Práctica Calificada 3

##### 2. Determinar el grafo de adyacencia del siguiente LO


\begin{align*}
\max \quad & x_1 + x_2 \\
\text { s.t. } \quad  &x_1& + 2 x_2& + x_{3}& {} {}& {} {}&=5&, \\
                      &x_1& - x_2&  {} {}& + x_{4}& {} {}&=2&, \\
                     &-x_1& + 3 x_2&  {} {}& {} {}& + x_{5}&=0&, \\
                     &x_1 \ge 0, x_2 \ge 0, x_3 \ge 0, x_4& \geq 0, x_{3} \geq 0
\end{align*}


In [ ]:
A = [[1, 2., 1, 0, 0],
     [1, -1, 0, 1, 0],
     [-1, 3, 0, 0, 1]]

b = [5, 2, 0.]

c = [1, 1, 0, 0, 0.]

tableau = build_tableau(A, b, c)

basic_var_list = list(map(lambda x: list(x), list(combinations(list(range(5)), 3))))
for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     print(f"x_B = {real_index(basic_var)}\n{tableau}")
     tableau = correct_all_tableau(tableau, basic_var)
     non_cycle_adjacent_graph(tableau, basic_var, [])

------- New simplex proccess -------
x_B = [1, 2, 3]
[[ 1.  1.  0.  0.  0.  0.]
 [ 1.  2.  1.  0.  0.  5.]
 [ 1. -1.  0.  1.  0.  2.]
 [-1.  3.  0.  0.  1.  0.]]
Full tableau corrected
vertex = [3. 1.]	x_B = [1, 2, 3]
[[ 0.   0.   0.  -2.  -1.  -4. ]
 [ 1.   0.   0.   1.5  0.5  3. ]
 [ 0.   1.   0.   0.5  0.5  1. ]
 [-0.  -0.   1.  -2.5 -1.5 -0. ]]
end proccess
------- New simplex proccess -------
x_B = [1, 2, 4]
[[ 1.  1.  0.  0.  0.  0.]
 [ 1.  2.  1.  0.  0.  5.]
 [ 1. -1.  0.  1.  0.  2.]
 [-1.  3.  0.  0.  1.  0.]]
Full tableau corrected
vertex = [3. 1.], x_B = [1, 2, 4]
[[ 0.   0.  -0.8  0.   0.2 -4. ]
 [ 1.   0.   0.6  0.  -0.4  3. ]
 [ 0.   1.   0.2  0.   0.2  1. ]
 [ 0.   0.  -0.4  1.   0.6  0. ]]
vertex = [3. 1.]	x_B = [1, 2, 5]
[[ 0.          0.         -0.66666667 -0.33333333  0.         -4.        ]
 [ 1.          0.          0.33333333  0.66666667  0.          3.        ]
 [ 0.          1.          0.33333333 -0.33333333  0.          1.        ]
 [ 0.          0.         